In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
%matplotlib inline

In [3]:
pd.pandas.set_option('display.max_columns', None)

In [4]:
from pickle import load
model1 = load(open('model1.pkl', 'rb'))
model2 = load(open('model2.pkl', 'rb'))
sc = load(open('scaler-1.pkl', 'rb'))
sc2 = load(open('scaler-2.pkl', 'rb'))

In [5]:
test = pd.read_csv('test.csv')

columns_with_na = [columns for columns in test.columns if test[columns].isnull().sum() > 1]
categoricals_na = [column for column in test.columns if test[column].isnull().sum() >= 1 and test[column].dtype == 'O']
numerical_columns = [column for column in test.columns if test[column].dtype != 'O']
columns_with_year = [column for column in test.columns if 'Yr' in column or 'Year' in column]
discrete_variables = [col for col in numerical_columns if len(test[col].unique()) < 25 and col not in columns_with_year + ['Id']]
cont_variables = [col for col in numerical_columns if len(test[col].unique()) > 25 and col not in columns_with_year + ['Id']]
categorical_columns = [column for column in test.columns if test[column].dtype == 'O']
numericals_na = [column for column in test.columns if test[column].isnull().sum() >= 1 and test[column].dtype != 'O']

In [6]:
for categorical_na in categoricals_na:
    test[categorical_na] = test[categorical_na].fillna(test[categorical_na].value_counts().index[0])

In [7]:
test[categoricals_na].isnull().sum()

MSZoning        0
Alley           0
Utilities       0
Exterior1st     0
Exterior2nd     0
MasVnrType      0
BsmtQual        0
BsmtCond        0
BsmtExposure    0
BsmtFinType1    0
BsmtFinType2    0
KitchenQual     0
Functional      0
FireplaceQu     0
GarageType      0
GarageFinish    0
GarageQual      0
GarageCond      0
PoolQC          0
Fence           0
MiscFeature     0
SaleType        0
dtype: int64

In [8]:
for numerical_na in numericals_na:
    test[numerical_na] = test[numerical_na].fillna(test[numerical_na].median())

In [9]:
test[numericals_na].isnull().sum()

LotFrontage     0
MasVnrArea      0
BsmtFinSF1      0
BsmtFinSF2      0
BsmtUnfSF       0
TotalBsmtSF     0
BsmtFullBath    0
BsmtHalfBath    0
GarageYrBlt     0
GarageCars      0
GarageArea      0
dtype: int64

In [10]:
for column_with_year in columns_with_year:
    if column_with_year != 'YrSold':
        test[column_with_year + '_YrSold'] = test['YrSold'] - test[column_with_year]
        
        test.drop([column_with_year], axis=1, inplace=True)
        numerical_columns.remove(column_with_year)
        numerical_columns.append(column_with_year + '_YrSold')

In [ ]:
import scipy.stats as stats

for cont_variable in cont_variables:
    if 0 not in dataset[cont_variable].unique():
        dataset[cont_variable], param = stats.boxcox(dataset[cont_variable]+1)

In [11]:
# from scipy.stats import skew

# skewed = test[cont_variables].apply(lambda x: skew(x.dropna().astype(float)))
# skewed = skewed[skewed > 0.75]
# skewed = skewed.index

# test[skewed] = np.log1p(test[skewed])

In [12]:
# for cont_variable in cont_variables:
#     if 0 not in test[cont_variable].unique():
#         test[cont_variable] = np.log(test[cont_variable])

In [13]:
numerical_columns.remove('Id')

categorical_columns_encoded = pd.get_dummies(test[categorical_columns])

# test_data = pd.concat([test[numerical_columns], categorical_columns_encoded], axis=1)

In [14]:
numerical_df = pd.DataFrame(sc.transform(test[numerical_columns]), columns=numerical_columns)

In [15]:
X_test = pd.concat([numerical_df, categorical_columns_encoded], axis=1)

In [16]:
# X_test = pd.DataFrame(sc.transform(test_data), columns=test_data.columns)

In [17]:
X_train = pd.read_csv('X_train.csv')
# feature_sel_model = load(open('feature.pkl', 'rb'))

In [18]:
# selected_feat = X_train.columns[(feature_sel_model.get_support())]

In [19]:
# X_test = X_test[selected_feat]

In [20]:
y1 = model1.predict(X_test)
y2 = model2.predict(X_test)

In [21]:
y1 = sc2.inverse_transform(model1.predict(X_test).reshape(-1,1))

In [22]:
y2 = sc2.inverse_transform(model2.predict(X_test).reshape(-1,1))

In [23]:
y_test = (y1 + y2) / 2

In [24]:
y_test = np.exp(y_test)

In [25]:
y_test.shape

(1459, 1)

In [26]:
original_dataset = pd.read_csv('test.csv')

sample_submission = pd.DataFrame({'Id': original_dataset['Id'], 'SalePrice': y_test.reshape(1, -1)[0]})

sample_submission.to_csv('sample_submission.csv', index=False)